In [38]:
!git clone https://github.com/Hitika-Jain/LegalTalk.git

%cd LegalTalk
#

Cloning into 'LegalTalk'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 49 (delta 17), reused 17 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 424.28 KiB | 4.88 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/LegalTalk/LegalTalk


In [ ]:
!pip install pandas
import re
import json
import csv
from pathlib import Path

In [ ]:
!pip install pymupdf nltk


In [ ]:
!ls -R .

.:
data			 notebooks	   src
graph-based-model.ipynb  requirements.txt  Untitled21.ipynb

./data:
case.pdf  statutes.csv

./notebooks:
ocr_regex.ipynb  sample.ipynb

./src:
app.py		   __init__.py		  statute_eda.py       utils.py
bert_mapper.py	   legal_case_adapter.py  statute_ensemble.py
explainability.py  preprocessing.py	  summaries.py
graph_mapper.py    regex_mapper.py	  summarizer.py


In [ ]:
import sys
sys.path.append('/content/LegalTalk')

# verify
sys.path[-1]

'/content/LegalTalk'

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [37]:
!pip install pymupdf

In [39]:
from src.preprocessing import parse_pdf_to_sentences

records = parse_pdf_to_sentences(
    "/content/Ntpc_Limited_vs_Jindal_Itf_Limited_Anr_on_30_January_2025.PDF",
    out_path="/content/casetext.jsonl"
)

len(records), records[:3]

[Saved] 1389 lines → /content/casetext.jsonl


(1389,
 [{'page': 0,
   'line_id': 0,
   'start_char': 0,
   'end_char': 41,
   'text': 'Ntpc Limited vs Jindal Itf Limited & Anr.'},
  {'page': 0,
   'line_id': 1,
   'start_char': 42,
   'end_char': 407,
   'text': 'on 30 January, 2025\nAuthor: Dinesh Kumar Sharma\nBench: Dinesh Kumar Sharma\n                          *      IN THE HIGH COURT OF DELHI AT NEW DELHI\n                                                                          RESERVED ON -0\n                          %                                          PRONOUNCED ON -30.0\n                          +      O.M.P.'},
  {'page': 0,
   'line_id': 2,
   'start_char': 408,
   'end_char': 429,
   'text': '(COMM) 204/2019, I.A.'}])

In [44]:

from src.preprocessing import parse_pdf_to_sentences

records = parse_pdf_to_sentences(
    "/content/drive/MyDrive/legal_dataset/Purushottam_Dashrath_Borate_Anr_vs_State_Of_Maharashtra_on_8_May_2015.PDF",
    out_path="/content/casetext.jsonl"
)

len(records), records[:3]

[Saved] 246 lines → /content/casetext.jsonl


(246,
 [{'page': 0,
   'line_id': 0,
   'start_char': 0,
   'end_char': 296,
   'text': 'Purushottam Dashrath Borate & Anr vs State Of Maharashtra on\n8 May, 2015\nEquivalent citations: AIR 2015 SUPREME COURT 2170, 2015 AIR SCW 3061,\nAIR 2015 SC (CRIMINAL) 1136, 2015 (2) ABR (CRI) 508, (2015) 6 SCALE 204,\n2015 ALLMR(CRI) 2421, (2015) 2 MAD LJ(CRI) 747, (2015) 3 CRIMES 114\nAuthor: H.L.'},
  {'page': 0,
   'line_id': 1,
   'start_char': 297,
   'end_char': 339,
   'text': 'Dattu\nBench: Arun Mishra, S.A. Bobde, H.L.'},
  {'page': 0,
   'line_id': 2,
   'start_char': 340,
   'end_char': 574,
   'text': 'Dattu\n                                                                   REPORTABLE\n                        IN THE SUPREME COURT OF INDIA\n                       CRIMINAL APPELLATE JURISDICTION\n                      CRIMINAL APPEAL No.'}])

In [47]:
# The error 'AttributeError: module 'src.regex_mapper' has no attribute 'map_text_to_hits''
# means that the function 'map_text_to_hits' does not exist in the 'src.regex_mapper' module.
# We need to inspect the 'src/regex_mapper.py' file to find the correct function name.
# Let's view the content of the file.
!cat src/regex_mapper.py

# -*- coding: utf-8 -*-
"""regex_mapper.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/16p1-RLac6aOYI1wLGkeN3IvcTi_QoMnW
"""

import re
import json
import csv
from pathlib import Path

# 1. Build improved regex patterns
# =============================

def build_statute_patterns():
    """
    Build regex patterns to extract statute references like:
      302
      376(2)(g)
      120-B
      120B
      234(b), 234-b, 234b
    without truncation.
    """

    flags = re.IGNORECASE | re.UNICODE

    # Full statute-id pattern
    STATUTE_ID = r'[0-9]{1,4}(?:\([0-9A-Za-z]+\)|-[A-Za-z]|[A-Za-z]){0,3}'

    patterns = []

    # Section ... pattern
    patterns.append((
        "section_keyword",
        re.compile(
            rf'\b(?:section|sections|sec|secs|s|§)\.?\s*[:\s]*({STATUTE_ID})',
            flags
        )
    ))

    # IPC prefix
    patterns.append((
        "ipc_prefix",
        re.compile(
            rf'\b

In [52]:
import json, re, sys, os
from pathlib import Path
import importlib # Added for module reloading

# --- Start of fix for src/regex_mapper.py content ---
regex_mapper_file_path = Path("src/regex_mapper.py")
if regex_mapper_file_path.exists():
    with open(regex_mapper_file_path, 'r', encoding='utf-8') as f:
        content_lines = f.readlines()

    # Filter out problematic Colab-specific lines
    cleaned_lines = []
    # Keywords to identify lines that should be removed from the module file
    problematic_keywords = [
        "from google.colab import drive",
        "drive.mount(",
        "!git clone",
        "%cd LegalTalk"
    ]

    for line in content_lines:
        if not any(kw in line for kw in problematic_keywords):
            cleaned_lines.append(line)

    # Write the cleaned content back to the file
    with open(regex_mapper_file_path, 'w', encoding='utf-8') as f:
        f.writelines(cleaned_lines)
    print(f"Cleaned {regex_mapper_file_path.name} by removing Colab-specific commands.")

# --- End of fix ---

# Update these paths if different
JSONL = "/content/casetext.jsonl"   # path created by your parser
# Or try repo-local copy
if not Path(JSONL).exists():
    if Path("data/case_sentences.jsonl").exists():
        JSONL = "data/case_sentences.jsonl"
    elif Path("data/sample_sentences.jsonl").exists():
        JSONL = "data/sample_sentences.jsonl"
print("Using JSONL:", JSONL, "exists?", Path(JSONL).exists())

# 1) Print first N records from JSONL to confirm parser output
def show_first_records(jsonl_path, n=20):
    print(f"\n--- Showing up to {n} parsed records ---\n")
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n:
                break
            try:
                rec = json.loads(line.strip())
            except Exception as e:
                print("JSON parse error on line", i, e)
                print(line)
                continue
            print(f"[{i}] page={rec.get('page')} line_id={rec.get('line_id')} chars=({rec.get('start_char')},{rec.get('end_char')}))")
            txt = rec.get('text','').replace('\n',' ')
            # print a short preview
            print("   >", txt[:300])
            print()
    print("...done\n")

if Path(JSONL).exists():
    show_first_records(JSONL, n=30)
else:
    raise FileNotFoundError(f"JSONL file not found: {JSONL}")

# 2) Show the patterns we are using (from regex_mapper.build_statute_patterns)

# --- Start of new fix: Ensure fresh import of regex_mapper ---
# Remove from sys.modules if already loaded to force a fresh import
if 'src.regex_mapper' in sys.modules:
    del sys.modules['src.regex_mapper']
# Also remove the parent package 'src' if it was loaded, to avoid caching issues
if 'src' in sys.modules:
    del sys.modules['src']
# Now import it freshly
from src import regex_mapper
# --- End of new fix ---

# The importlib.reload line is now removed as it is redundant after del and fresh import.

patterns = regex_mapper.build_statute_patterns()
print("\n--- Compiled Patterns ---")
for name, pat in patterns:
    print(name, "->", pat.pattern)
print()

# 3) Run the matcher on each record and collect any matches
print("\n--- Running matcher over parsed sentences (first 200) ---\n")
matches_total = 0
no_match_but_keywords = []
with open(JSONL, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 200:
            break
        rec = json.loads(line)
        text = rec.get('text','') or ''
        # This is the line that caused the AttributeError, it should work after cleaning and reloading
        found = regex_mapper.find_statutes_in_text(text, patterns=patterns)
        if found:
            matches_total += len(found)
            # Corrected m['statute_norm'] to m['statute_raw'] based on find_statutes_in_text return structure
            for m in found:
                print(f"[hit] line#{i} page={rec.get('page')} match={m['match_text']} -> raw={m['statute_raw']} pattern={m['pattern_name']}")
        else:
            # record sentences that contain likely keywords for further inspection
            low = text.lower()
            if 'section' in low or 'sec.' in low or 's.' in low or 'ipc' in low or 'article' in low or re.search(r'\b[0-9]{2,3}\b', low):
                no_match_but_keywords.append((i, text[:200]))
print("\nTotal matches found (first 200 lines):", matches_total)
print("Sentences with keywords but no match (sample up to 20):", len(no_match_but_keywords))
for idx, t in no_match_but_keywords[:20]:
    print(f" - line#{idx}: {t}")
print()

# 4) Fallback: run extremely broad patterns (debug only)
print("\n--- Fallback broad checks (debug) ---")
broad_patterns = {
    "section_simple": re.compile(r'\bsection\s+[:\s]*([0-9]{1,4}[A-Za-z\-]*)', re.IGNORECASE),
    "s_dot": re.compile(r'\bs\.\s*([0-9]{1,4}[A-Za-z\-]*)', re.IGNORECASE),
    "ipc_then_num": re.compile(r'\bipc[\s_:]*([0-9]{1,4}[A-Za-z\-]*)', re.IGNORECASE),
    "num_ipc_after": re.compile(r'\b([0-9]{1,4}[A-Za-z\-]*)\s*(?:,?\s*ipc)\b', re.IGNORECASE),
    "bare_num_letter": re.compile(r'\b([0-9]{2,3}[A-Za-z]?)\b')
}
broad_matches = 0
with open(JSONL, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 200:
            break
        rec = json.loads(line)
        text = rec.get('text','') or ''
        for pname, p in broad_patterns.items():
            m = p.search(text)
            if m:
                broad_matches += 1
                print(f"[broad:{pname}] line#{i} -> {m.group(0)} -> group1={m.group(1) if m.groups() else None}")
                break
print("Broad matches found (first 200):", broad_matches)

# 5) If 0 matches, produce a CSV report of sentences containing keywords for manual inspection
if matches_total == 0:
    import csv
    out_csv = "/content/regex_debug_keyword_lines.csv"
    with open(JSONL, 'r', encoding='utf-8') as fin, open(out_csv, 'w', newline='', encoding='utf-8') as fout:
        writer = csv.writer(fout)
        writer.writerow(["line_no", "page", "line_id", "text_snippet"])
        for i, line in enumerate(fin):
            rec = json.loads(line)
            text = rec.get('text','') or ''
            low = text.lower()
            if 'section' in low or 'sec.' in low or 's.' in low or 'ipc' in low or 'article' in low or re.search(r'\b[0-9]{2,3}\b', low):
                writer.writerow([i, rec.get('page'), rec.get('line_id'), text[:300].replace('\n',' ')])
    print(f"Wrote keyword lines report to {out_csv}. Please inspect this CSV to see why regex may have failed.")
else:
    print("Matches detected — no keyword report generated.")

Cleaned regex_mapper.py by removing Colab-specific commands.
Using JSONL: /content/casetext.jsonl exists? True

--- Showing up to 30 parsed records ---

[0] page=0 line_id=0 chars=(0,296))
   > Purushottam Dashrath Borate & Anr vs State Of Maharashtra on 8 May, 2015 Equivalent citations: AIR 2015 SUPREME COURT 2170, 2015 AIR SCW 3061, AIR 2015 SC (CRIMINAL) 1136, 2015 (2) ABR (CRI) 508, (2015) 6 SCALE 204, 2015 ALLMR(CRI) 2421, (2015) 2 MAD LJ(CRI) 747, (2015) 3 CRIMES 114 Author: H.L.

[1] page=0 line_id=1 chars=(297,339))
   > Dattu Bench: Arun Mishra, S.A. Bobde, H.L.

[2] page=0 line_id=2 chars=(340,574))
   > Dattu                                                                    REPORTABLE                         IN THE SUPREME COURT OF INDIA                        CRIMINAL APPELLATE JURISDICTION                       CRIMINAL APPEAL No.

[3] page=0 line_id=3 chars=(575,622))
   > 1439 OF 2013 PURUSHOTTAM DASHRATH BORATE & ANR.

[4] page=0 line_id=4 chars=(633,799))
   > …Appell

In [53]:
# DIAGNOSTIC: test patterns on one sentence
import json
from pathlib import Path
from src import regex_mapper

JSONL = "/content/casetext.jsonl"
# pick a specific line that contained "Section 120-B" (from your report it was line#8 / line#70 / etc.)
target_indices = [8, 70, 99, 103]  # try several lines that the fallback reported
records = []
with open(JSONL, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i in target_indices:
            records.append((i, json.loads(line)))

patterns = regex_mapper.build_statute_patterns()
print("Testing compiled patterns:\n")
for idx, rec in records:
    text = rec.get('text','')
    print(f"--- line#{idx} preview: {text[:240]!r} ---")
    for name, pat in patterns:
        m = pat.search(text)
        print(f"{name:20s} -> match? {bool(m)}", f" | matched: {m.group(0)!r}" if m else "")
    print()

Testing compiled patterns:

--- line#8 preview: 'The Prosecution case in a nutshell is:\nThe deceased was residing with her brother-in-law and sister, namely PW-12 and\nPW-13 respectively, along with their minor son, in a flat in Pune City.' ---
section_keyword      -> match? False 
ipc_prefix           -> match? False 
section_of_ipc       -> match? False 
num_then_ipc         -> match? False 
article              -> match? False 
bare_number_with_letter -> match? False 

--- line#70 preview: 'by way\nof strangulation.' ---
section_keyword      -> match? False 
ipc_prefix           -> match? False 
section_of_ipc       -> match? False 
num_then_ipc         -> match? False 
article              -> match? False 
bare_number_with_letter -> match? False 

--- line#99 preview: 'The\nHigh Court has observed that the heinous acts have been committed by the accused- appellants in a\ndiabolical and cold-blooded manner without any hesitation and undeterred by its consequences.' ---
section_keyw

In [55]:
# === Debug + robust regex mapper update ===
import json, re, sys, os
from pathlib import Path

# 0) paths — adjust if needed
JSONL = "/content/casetext.jsonl"   # your parser output
OUT_CSV = "/content/regex_hits_debug.csv"

print("JSONL exists?", Path(JSONL).exists())
if not Path(JSONL).exists():
    raise FileNotFoundError(JSONL)

# 1) Inspect first record keys and a sample text (raw repr to reveal invisible chars)
with open(JSONL, 'r', encoding='utf-8') as f:
    first = json.loads(next(f))
print("\nFirst record keys:", list(first.keys()))
sample_text = first.get('text') or first.get('sentence') or first.get('line') or first.get('content')
print("\nSample text (repr):\n", repr(sample_text)[:500])
print("\nSample text (print):\n", sample_text[:500])

# Show character codes of first 120 chars to discover NBSP or zero-width
print("\nChar codes (first 120 chars):")
for i, ch in enumerate(sample_text[:120]):
    print(i, ch, hex(ord(ch)))

# 2) Load the regex code (we'll create a robust local copy for immediate use)
def build_patterns():
    flags = re.IGNORECASE | re.UNICODE
    gap = r'[\s\u00A0\-:\.]*'
    patterns = [
        ("section_keyword", re.compile(r'\b(?:section|sec|s|secs|§)'+ gap + r'([0-9]{1,4}[A-Za-z\-\/]{0,10})\b', flags)),
        ("ipc_prefix", re.compile(r'\b(?:ipc|indian\s+penal\s+code)'+ gap + r'[^\w]{0,3}'+ gap + r'([0-9]{1,4}[A-Za-z\-\/]{0,10})\b', flags)),
        ("num_then_ipc", re.compile(r'\b([0-9]{1,4}[A-Za-z\-\/]{0,10})'+ gap + r'(?:,?\s*ipc|,?\s*indian\s+penal\s+code)\b', flags)),
        ("section_of_ipc", re.compile(r'\bsection'+ gap + r'([0-9]{1,4}[A-Za-z\-\/]{0,10})'+ gap + r'(?:of|under)'+ gap + r'(?:the\s*)?(?:ipc|indian\s+penal\s+code)\b', flags)),
        ("article", re.compile(r'\b(?:article|art)'+ gap + r'([0-9]{1,3}[A-Za-z\-\/]{0,6})\b', flags)),
        ("section_ipc_anywhere", re.compile(r'\bsection'+ gap + r'([0-9]{1,4}[A-Za-z\-\/]{0,10}).{0,20}?(?:ipc|indian\s+penal\s+code)\b', flags)),
        ("bare_number_with_letter", re.compile(r'\b([0-9]{1,4}[A-Za-z])\b', flags))
    ]
    return patterns

_zero_width_chars = ['\u200b', '\u200c', '\u200d', '\ufeff']

def normalize_text_for_matching(s: str):
    if s is None:
        return ""
    s2 = s.replace('\u00A0', ' ')  # NBSP -> space
    for z in _zero_width_chars:
        s2 = s2.replace(z, '')
    # collapse weird multi-space runs
    s2 = re.sub(r'\s+', ' ', s2)
    return s2

# 4) Function to find matches on normalized text then map back to original indices
def find_matches_map_to_original(text: str, patterns=None):
    patterns = patterns or build_patterns()
    orig = text or ""
    norm = normalize_text_for_matching(orig)
    results = []
    # We'll keep track of a cursor in original text to find subsequent matches to avoid wrong repeats
    orig_cursor = 0
    for name, pat in patterns:
        for m in pat.finditer(norm):
            matched_norm = m.group(0)
            raw_group = m.group(1) if m.groups() else None
            # Try to find the matched_norm in the original text starting from orig_cursor
            # but matched_norm may have normalized spaces; try a few strategies:
            idx = orig.find(matched_norm, orig_cursor)
            if idx == -1:
                # strategy B: locate using raw_group if exists
                if raw_group:
                    # raw_group likely preserved (digits/letters). try to find raw_group in orig
                    idx = orig.find(raw_group, orig_cursor)
                    if idx != -1:
                        # compute match_end by adding len(raw_group) in orig
                        idx_end = idx + len(raw_group)
                        match_text_orig = orig[idx:idx_end]
                        statute_raw = raw_group
                    else:
                        # fallback: fuzzy by stripping spaces from both and search (last resort)
                        try:
                            mn_norm_nospace = re.sub(r'\s+', '', matched_norm)
                            orig_nospace = re.sub(r'\s+', '', orig)
                            pos_nospace = orig_nospace.find(mn_norm_nospace)
                            if pos_nospace != -1:
                                # estimate original index by mapping positions (approx)
                                # find approximate index by searching substring of length 10
                                glimpse = mn_norm_nospace[:10]
                                idx = orig.find(glimpse, orig_cursor)
                        except Exception:
                            idx = -1
                else:
                    idx = -1
            if idx == -1:
                # give up mapping to original for this match (skip)
                # but record debug info
                results.append({
                    'pattern': name,
                    'matched_normalized': matched_norm,
                    'raw_group': raw_group,
                    'mapped': False,
                    'orig_index': None,
                    'orig_match_text': None
                })
                continue
            # Success mapping to original
            # compute end based on matched_norm length vs original substring: try to match same length
            end = idx + len(matched_norm)
            # But matched_norm may have collapsed spaces; prefer taking the span that corresponds to raw_group if present
            if raw_group:
                # find raw_group starting at idx or after
                rg_idx = orig.find(raw_group, idx)
                if rg_idx != -1 and rg_idx >= idx and rg_idx < end + 20:
                    idx = rg_idx
                    end = rg_idx + len(raw_group)
            match_text_orig = orig[idx:end]
            results.append({
                'pattern': name,
                'matched_normalized': matched_norm,
                'raw_group': raw_group,
                'mapped': True,
                'orig_index': idx,
                'orig_end': end,
                'orig_match_text': match_text_orig
            })
            # move cursor to end to avoid duplicate finds
            orig_cursor = end
    return results

# 5) Run debug on first N records to show per-record pattern behavior
N = 50
hits = []
with open(JSONL, 'r', encoding='utf-8') as fin:
    for i, line in enumerate(fin):
        if i >= N: break
        rec = json.loads(line)
        text = rec.get('text') or rec.get('sentence') or rec.get('line') or rec.get('content') or ''
        res = find_matches_map_to_original(text, patterns=patterns)
        # collect only mapped successes
        mapped = [r for r in res if r.get('mapped')]
        if mapped:
            print(f"\n=== LINE #{i} page={rec.get('page')} line_id={rec.get('line_id')} ===")
            print("ORIG repr:", repr(text[:200]))
            for m in mapped:
                print("pattern:", m['pattern'], "orig_index:", m['orig_index'], "orig_end:", m['orig_end'], "match_text:", repr(m['orig_match_text']))
            hits.append((i, mapped))
print("\nTOTAL LINES with mapped hits in first", N, "=", len(hits))

# 6) If mapped hits found, write a CSV using the robust mapper logic (global offsets if start present)
if len(hits) > 0:
    import csv
    out_rows = []
    with open(JSONL, 'r', encoding='utf-8') as fin:
        for i, line in enumerate(fin):
            rec = json.loads(line)
            text = rec.get('text') or rec.get('sentence') or rec.get('line') or rec.get('content') or ''
            page = rec.get('page')
            line_id = rec.get('line_id')
            sent_start = rec.get('start_char') or rec.get('start') or rec.get('char_start') or 0
            for m in find_matches_map_to_original(text, patterns=patterns):
                if not m.get('mapped'):
                    continue
                out_rows.append({
                    "line_no": i,
                    "page": page,
                    "line_id": line_id,
                    "sentence_text": text,
                    "match_text": m['orig_match_text'],
                    "match_start": m['orig_index'],
                    "match_end": m['orig_end'],
                    "global_start": (sent_start + m['orig_index']) if isinstance(sent_start, int) else None,
                    "global_end": (sent_start + m['orig_end']) if isinstance(sent_start, int) else None,
                    "raw_group": m['raw_group'],
                    "pattern": m['pattern']
                })
    # write CSV
    with open(OUT_CSV, 'w', encoding='utf-8', newline='') as fout:
        w = csv.DictWriter(fout, fieldnames=list(out_rows[0].keys()))
        w.writeheader()
        for r in out_rows:
            w.writerow(r)
    print("Wrote debug CSV:", OUT_CSV, "rows:", len(out_rows))
else:
    print("No mapped matches found in first", N, "records. Please paste one example sentence that definitely contains 'Section' or 'IPC' (copy its exact text) and I will craft a custom pattern.")

JSONL exists? True

First record keys: ['page', 'line_id', 'start_char', 'end_char', 'text']

Sample text (repr):
 'Purushottam Dashrath Borate & Anr vs State Of Maharashtra on\n8 May, 2015\nEquivalent citations: AIR 2015 SUPREME COURT 2170, 2015 AIR SCW 3061,\nAIR 2015 SC (CRIMINAL) 1136, 2015 (2) ABR (CRI) 508, (2015) 6 SCALE 204,\n2015 ALLMR(CRI) 2421, (2015) 2 MAD LJ(CRI) 747, (2015) 3 CRIMES 114\nAuthor: H.L.'

Sample text (print):
 Purushottam Dashrath Borate & Anr vs State Of Maharashtra on
8 May, 2015
Equivalent citations: AIR 2015 SUPREME COURT 2170, 2015 AIR SCW 3061,
AIR 2015 SC (CRIMINAL) 1136, 2015 (2) ABR (CRI) 508, (2015) 6 SCALE 204,
2015 ALLMR(CRI) 2421, (2015) 2 MAD LJ(CRI) 747, (2015) 3 CRIMES 114
Author: H.L.

Char codes (first 120 chars):
0 P 0x50
1 u 0x75
2 r 0x72
3 u 0x75
4 s 0x73
5 h 0x68
6 o 0x6f
7 t 0x74
8 t 0x74
9 a 0x61
10 m 0x6d
11   0x20
12 D 0x44
13 a 0x61
14 s 0x73
15 h 0x68
16 r 0x72
17 a 0x61
18 t 0x74
19 h 0x68
20   0x20
21 B 0x42
22 o 0x6f
23 r 0x72


In [56]:
# Run mapper and write CSV
from src import regex_mapper
jsonl_path = "/content/casetext.jsonl"   # update if your file is different
out_csv = "/content/regex_hits_with_global_offsets.csv"

rows = regex_mapper.map_jsonl_file_to_hits(jsonl_path, out_csv, include_legal_terms=True)
print("Total hits:", len(rows))

Wrote 35 hits → /content/regex_hits_with_global_offsets.csv
Total hits: 35


In [59]:
!pip install fuzzywuzzy

In [64]:
import re
from fuzzywuzzy import fuzz, process  # keep if you still want fuzzy as last resort

# canonical_ids = set(canon['id'].astype(str).tolist())  # you already have this

def normalize_canonical_id(raw: str):
    """
    Convert raw strings such as:
      '120-B', '120b', 'Section 120-B',
      '376(2)(g)', '376 (2) (g)'
    into canonical-ish IDs like:
      'IPC_120B', 'IPC_376_2_G'
    WITHOUT dropping subsection info.
    """
    if not raw:
        return None

    s = raw.upper().strip()

    # strip leading words like 'SECTION 376(2)(G)' → '376(2)(G)'
    s = re.sub(r'^(SECTION|SECTIONS|SEC|SECS|S|SS)\.?\s*', "", s)

    # strip IPC prefix if present
    s = s.replace("IPC_", "").replace("IPC", "").strip()

    # normalize "120-B" → "120B" (number-letter hyphen)
    s = re.sub(r'(?<=\d)-(?=[A-Z])', "", s)

    # If we have parentheses → treat as subsection: 376(2)(g) → 376_2_G
    if "(" in s:
        s2 = re.sub(r'\((\d+)\)', r'_\1', s)      # (2) → _2
        s2 = re.sub(r'\(([A-Z])\)', r'_\1', s2)   # (G) → _G
        s2 = s2.replace(" ", "")
        return "IPC_" + s2

    # Simple forms: 120, 120B, 302 etc.
    if re.match(r'^\d{1,4}[A-Z]?$', s):
        return "IPC_" + s

    # Fallback: just glue IPC_ in front of whatever cleaned thing we got
    s = s.replace(" ", "")
    return "IPC_" + s


def map_to_canonical(stat_norm: str, canon_df=None):
    """
    Convert normalized regex match (e.g., '120-B', '376(2)(g)')
    to a canonical ID.

    **Key rule**: DO NOT silently collapse subsections like
    '376(2)(g)' → 'IPC_376'. Keep them as 'IPC_376_2_G', even if
    they are not present in statutes.csv.
    """
    if not stat_norm or stat_norm.strip() == "":
        return None

    raw = stat_norm.strip()
    candidate = normalize_canonical_id(raw)

    # 1) Exact match in canonical list → use it.
    if candidate in canonical_ids:
        return candidate

    # 2) If the raw string clearly contains subsection info
    #    (parentheses, hyphens, or trailing letter), KEEP candidate as-is.
    if re.search(r'[\(\)\-]', raw) or re.match(r'^\d{1,4}[A-Z]$', raw):
        return candidate

    # 3) If it's just a bare number like '302', try mapping to IPC_302
    just_digits = re.match(r'^(\d{1,4})$', raw)
    if just_digits:
        basic = f"IPC_{just_digits.group(1)}"
        if basic in canonical_ids:
            return basic
        # if not present, still return candidate
        return candidate

    # 4) Optional: fuzzy match ONLY for cases without subsection info
    #    (to recover from OCR noise like '3O2' vs '302')
    #    You can remove this block if you don't want fuzzy at all.
    if canon_df is not None:
        best = process.extractOne(candidate, list(canonical_ids), scorer=fuzz.token_set_ratio)
        if best and best[1] > 92:
            return best[0]

    # 5) Last resort: keep the detailed candidate
    return candidate

# ---------- Build weak-label dataset ----------
grouped = defaultdict(set)
meta = {}

for _, row in hits.iterrows():
    key = f"{row.get('page','')}__{row.get('line_id','')}"
    stat_norm = row.get('statute_normalized', '') or row.get('raw_group','')

    cid = map_to_canonical(stat_norm)

    if cid:
        grouped[key].add(cid)

    meta[key] = {
        'page': row.get('page',''),
        'line_id': row.get('line_id',''),
        'sentence_text': row.get('sentence_text','')
    }

# Build DataFrame
rows = []
for k, values in grouped.items():
    m = meta[k]
    rows.append({
        'sentence_id': k,
        'page': m['page'],
        'line_id': m['line_id'],
        'sentence_text': m['sentence_text'],
        'canonical_statutes': ";".join(sorted(values))
    })

weak_df = pd.DataFrame(rows)
weak_df.to_csv(OUT_WEAK, index=False)
print("Wrote weak_labels.csv →", len(weak_df), "rows")

# ---------- Optional multi-hot ----------
all_canon_sorted = sorted(list(canonical_ids))

for cid in all_canon_sorted:
    weak_df[cid] = weak_df['canonical_statutes'].apply(lambda s: 1 if (cid in str(s).split(";")) else 0)

weak_df.to_csv(OUT_MULTIHOT, index=False)
print("Wrote multihot:", OUT_MULTIHOT, "shape:", weak_df.shape)

# Show top rows
weak_df.head(10)

Wrote weak_labels.csv → 12 rows
Wrote multihot: /content/weak_labels_multihot.csv shape: (12, 459)


/tmp/ipython-input-3547468400.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weak_df[cid] = weak_df['canonical_statutes'].apply(lambda s: 1 if (cid in str(s).split(";")) else 0)
/tmp/ipython-input-3547468400.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weak_df[cid] = weak_df['canonical_statutes'].apply(lambda s: 1 if (cid in str(s).split(";")) else 0)
/tmp/ipython-input-3547468400.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

,sentence_id,page,line_id,sentence_text,canonical_statutes,IPC_107,IPC_109,IPC_114,IPC_120,IPC_120B,...,IPC_502,IPC_503,IPC_504,IPC_505,IPC_506,IPC_507,IPC_508,IPC_509,IPC_510,IPC_511
0,0__7,0,7,"By the impugned judgment and order,\nthe High ...",IPC_120B;IPC_1860;IPC_302,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2__4,2,4,"Therefore, an offence under Section 302 of the...",IPC_302,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3__1,3,1,"On 05.03.2009, the\ncharges were framed under ...",IPC_364,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3__2,3,2,"On 03.04.2010, the charge was altered and the ...",IPC_120B,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3__3,3,3,"Additionally, the charge under Section 120-B o...",IPC_120B;IPC_302,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,3__17,3,17,The Sessions Court has noticed that the eviden...,IPC_313,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4__5,4,5,After affording an\nopportunity of hearing to ...,IPC_120B;IPC_302;IPC_364;IPC_376_2_G;IPC_404,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,4__8,4,8,"Aggrieved by the aforesaid judgment and order,...",IPC_366,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5__16,5,16,"However, it would be apposite to notice the de...",IPC_302,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,7__3,7,3,"For instance,\nmurder is of a person who had a...",IPC_43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
